#### IMPORTO LIBRERIAS ####

In [338]:
import pandas as pd
import numpy as np
from sklearn import linear_model

#configuro notebook para mostrar varios salidas en la misma celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#amplio el output de columnas a 500
pd.set_option('display.max_columns', 500)

!pwd
!ls -l

/Users/mac/Documents/MASTER_DATA_SCIENCE/tfm
total 265352
-rw-r--r--   1 mac  staff    221629 12 jun 01:55 Analisis_DGT.ipynb
-rw-r--r--   1 mac  staff    104803 15 abr 11:08 TFM_DGT-pruebas.ipynb
-rw-r--r--   1 mac  staff    220491 12 jun 01:57 TFM_DGT.ipynb
-rw-r--r--   1 mac  staff  33726464 31 may 22:31 aa.db
drwxr-xr-x  21 mac  staff       714 31 may 22:43 cleaned
-rw-r--r--   1 mac  staff  33726464 31 may 14:35 data.db
-rw-r--r--   1 mac  staff      8192 31 may 13:43 datos_dgt.db
-rw-r--r--   1 mac  staff         0 23 may 22:42 gg
-rw-r--r--   1 mac  staff  67846144 31 may 22:27 jj.db
drwxr-xr-x   5 mac  staff       170 30 mar 23:28 metadata
drwxr-xr-x  11 mac  staff       374 30 mar 23:28 raw
drwxr-xr-x  15 mac  staff       510 12 jun 01:57 raw_zip


### CONSERVO LOS DATOS ORIGINALES EN LA CARPETA RAW. LOS FICHEROS DE TRABAJO ESTAN EN LA CARPETA CLEANED

In [339]:
!ls -l
!cp -r raw/* cleaned/

total 265352
-rw-r--r--   1 mac  staff    221629 12 jun 01:55 Analisis_DGT.ipynb
-rw-r--r--   1 mac  staff    104803 15 abr 11:08 TFM_DGT-pruebas.ipynb
-rw-r--r--   1 mac  staff    220491 12 jun 01:57 TFM_DGT.ipynb
-rw-r--r--   1 mac  staff  33726464 31 may 22:31 aa.db
drwxr-xr-x  21 mac  staff       714 31 may 22:43 cleaned
-rw-r--r--   1 mac  staff  33726464 31 may 14:35 data.db
-rw-r--r--   1 mac  staff      8192 31 may 13:43 datos_dgt.db
-rw-r--r--   1 mac  staff         0 23 may 22:42 gg
-rw-r--r--   1 mac  staff  67846144 31 may 22:27 jj.db
drwxr-xr-x   5 mac  staff       170 30 mar 23:28 metadata
drwxr-xr-x  11 mac  staff       374 30 mar 23:28 raw
drwxr-xr-x  15 mac  staff       510 12 jun 01:57 raw_zip


### GUARDO UN LISTADO POR TIPOLOGIA DE FICHERO, PARA POSTERIORMENTE ALMACENARLO EN UN DICCIONARIO ###

In [340]:
accvict=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_ACCVICT_*.csv
pers=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_PERS_*.csv
vehic=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_VEHIC_*.csv
dic_all_files={'Accvict':accvict,'Pers':pers,'Vehic':vehic}
len(dic_all_files)

3

#### RUTAS DE TODOS LOS FICHEROS ####

In [341]:
dic_all_files

{'Accvict': ['./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv',
  './cleaned/MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv',
  './cleaned/MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv',
  './cleaned/MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv',
  './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv',
  './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv',
  './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv',
  './cleaned/MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv'],
 'Pers': ['./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_PERS_24H_2008.csv',
  './cleaned/MICRODATOS_ACC_VICT_2009/TABLA_PERS_24H_2009.csv',
  './cleaned/MICRODATOS_ACC_VICT_2010/TABLA_PERS_24H_2010.csv',
  './cleaned/MICRODATOS_ACC_VICT_2011/TABLA_PERS_2011.csv',
  './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_PERS_2012.csv',
  './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_PERS_2013.csv',
  './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_PERS_2014.csv',
  './cleaned/MICRODATOS_ACC_VICT

### Ahora que los datos estan trazados, procedemos a realizar una observacion preliminar para comprobar si hay homgeneidad en los datos, y si disponemos de la misma informacion en cada data set, ya que segun la DGT, las extructuras de los datos cambian a partir del 2011 en hasta el 2015.

### Por los que tenemos dos grupos de datos:

### Tipo 1) 2008-2010
### Tipo 2) 2011-2015

### Recorro cada clave - para obtener las cabeceras y poder comparar que campos tengo en cada año por tipo de fichero, e introduzco los data frames en una lista.

In [342]:
dic_all_files.keys()

dict_keys(['Accvict', 'Pers', 'Vehic'])

In [343]:
#Creo una lista con las cabeceras de los ficheros
Headers_List=[]
for key in dic_all_files.keys():
    df_2008=pd.read_csv(dic_all_files[key][0],delimiter=';',nrows=0,encoding="ISO-8859-1")
    df_2008=df_2008.transpose()
    df_2008.reset_index(inplace=True)
    df_2008.columns=[key+'_2008']

    df_2015=pd.read_csv(dic_all_files[key][7],delimiter=';',nrows=0,encoding="ISO-8859-1")
    df_2015=df_2015.transpose()
    df_2015.reset_index(inplace=True)
    df_2015.columns=[key+'_2015']
        
    Headers_List.append(df_2008.merge(df_2015, left_on=key+'_2008', right_on=key+'_2015', how='outer'))

### --------------------------------------------------------------------------------------------------

## REVISION DE LOS TRES TIPOS DE DATA SETS

### SE DISPONE DE TRES TIPOS DE DATASETS:

#### 1. ACCIDENTES
#### 2. PERSONAS
#### 3 .VEHICULOS

#### COMO PRIMER PASO VOY A REVISAR TANTO ESTRUCTURA, PARA DECIDIR QUE VARIABLES MANTENDRE PARA EL ANALISIS POSTERIOR. PROCEDERE A MODIFICAR LOS DATASET DE LA CARPETA "./CLEANED" ANTES DE REALIZAR LA CARGA PARA EN ANALISIS.

### --------------------------------------------------------------------------------------------------

###   1. ACCIDENTES

In [344]:
#Cabecera del fichero de Accidentes_victimas
Header_Accvict=Headers_List[0]
Header_Accvict

,Accvict_2008,Accvict_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ANIO,ANIO
2,MES,MES
3,HORA,HORA
4,DIASEMANA,DIASEMANA
5,PROVINCIA,PROVINCIA
6,COMUNIDAD_AUTONOMA,COMUNIDAD_AUTONOMA
7,ISLA,ISLA
8,MUNICIPIO,MUNICIPIO
9,TOT_VICTIMAS,TOT_VICTIMAS


In [345]:
#mantengo solo las columnas que son iguales en los dos tipos de dataset

Header_Accvict=Header_Accvict[Header_Accvict['Accvict_2008']==Header_Accvict['Accvict_2015']]
Header_Accvict=Header_Accvict['Accvict_2008'].tolist()
dic_all_headers={'Accvict':Header_Accvict}
dic_all_headers

#Genero nuevo DF vacio con los nombres de columnas
df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Accvict'])

#Primero genero una vista con todos los datos para examinarlos
for file in dic_all_files['Accvict']:
    df_vict=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Accvict'],encoding="ISO-8859-1",dtype={'ID_ACCIDENTE': object})
    #relleno los nulos con 999
    df_vict.fillna(999,inplace=True)
    df_all_years=pd.concat([df_all_years,df_vict],ignore_index=True)


#muestreo los datos dde la vista
df_all_years.head(3)
df_all_years.tail(3)
df_all_years.sample(3)

#Decido descartar los siguientes campos, al no considerarlos relevantes o estar muy incompletos:
df_all_years.drop(['ISLA','MUNICIPIO','TIPO_INTERSEC','ACERAS','CARRETERA','TRAZADO_NO_INTERSEC'], axis=1, inplace=True)

#Actualizo mi diccionario con las cabeceras que utilizare en el proceso de carga
Header_Accvict=df_all_years.columns.tolist()
dic_all_headers={'Accvict':Header_Accvict}
dic_all_headers
#En el campo factores atmosfericos, como lo considero relevante
#y tengo 4787 valores desconocidos, englobo estos registros dentro de la categoria otros(9)

#Vacio el df de la vista, que contendra todos los años
df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Accvict'])

for file in dic_all_files['Accvict']:
    df_acc=pd.read_csv(file,delimiter=";",usecols=dic_all_headers['Accvict'],encoding="ISO-8859-1",dtype={'ID_ACCIDENTE': object})
    df_acc.fillna(999,inplace=True)

    #factores atmosfericos desconocidos,tal y como cometo arriba van a otros
    df_acc.ix[(df_acc['FACTORES_ATMOSFERICOS']==999),'FACTORES_ATMOSFERICOS'] = 9
    #df_acc.ix[(df_vict['VISIBILIDAD_RESTRINGIDA']==999), 'VISIBILIDAD_RESTRINGIDA'] = df_all_years['VISIBILIDAD_RESTRINGIDA'][df_all_years['VISIBILIDAD_RESTRINGIDA']!=999].median()
    
    #Recorro todas las columnas del df, sustituyendo los known por la mediana del dato para tipos != objeto 'O'
    for col in df_acc.columns:
        if df_acc[col].dtype !='O':
            df_acc.ix[(df_acc[col]==999), col]=df_acc[col][df_acc[col]!=999].median()
            df_acc[col]=df_acc[col].astype(int)
        
    df_all_years=pd.concat([df_all_years,df_acc],ignore_index=True)
    df_acc.to_csv(file,index=False,sep=";",encoding="ISO-8859-1")
    print("MODFICADO ----- "+file)

#Convierto tambien los numericos de la vista tipo float a enteros
#Compruebo que me ha sustituido correctamente los 999 por la mediana.
for col in df_all_years.columns:
    if df_all_years[col].dtype !='O':
        df_all_years[col]=df_all_years[col].astype(int)
    df_all_years[col].value_counts(dropna=False)

df_all_years.to_csv("./cleaned/view_accvict_08_15.csv",index=False,sep=";",encoding="ISO-8859-1")
print("CREADA VISTA CON DATOS DE ACCIDENTES Y VISTIMAS DE 2008 A 2015")

########   FIN DEL PROCESAMIENTO DE LOS DATASET DE ACCIDENTES #########



{'Accvict': ['ID_ACCIDENTE',
  'ANIO',
  'MES',
  'HORA',
  'DIASEMANA',
  'PROVINCIA',
  'COMUNIDAD_AUTONOMA',
  'ISLA',
  'MUNICIPIO',
  'TOT_VICTIMAS',
  'TOT_MUERTOS',
  'TOT_HERIDOS_GRAVES',
  'TOT_HERIDOS_LEVES',
  'TOT_VEHICULOS_IMPLICADOS',
  'ZONA',
  'ZONA_AGRUPADA',
  'CARRETERA',
  'RED_CARRETERA',
  'TIPO_VIA',
  'TRAZADO_NO_INTERSEC',
  'TIPO_INTERSEC',
  'SUPERFICIE_CALZADA',
  'LUMINOSIDAD',
  'FACTORES_ATMOSFERICOS',
  'VISIBILIDAD_RESTRINGIDA',
  'ACERAS',
  'TIPO_ACCIDENTE']}

,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,TOT_MUERTOS,TOT_HERIDOS_GRAVES,TOT_HERIDOS_LEVES,TOT_VEHICULOS_IMPLICADOS,ZONA,ZONA_AGRUPADA,CARRETERA,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
0,2008480000372,2008.0,4.0,7.0,5.0,48.0,17.0,999.0,999.0,4.0,0.0,0.0,4.0,1.0,1.0,1.0,BI-631,3.0,2.0,1.0,999.0,2.0,3.0,1.0,8.0,N,71.0
1,2008250021218,2008.0,3.0,14.0,7.0,25.0,9.0,999.0,999.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,C-12,2.0,5.0,2.0,999.0,1.0,1.0,9.0,8.0,N,22.0
2,2008430024095,2008.0,10.0,12.0,7.0,43.0,9.0,999.0,999.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,TV-7331,4.0,5.0,1.0,999.0,1.0,1.0,1.0,0.0,N,68.0


,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,TOT_MUERTOS,TOT_HERIDOS_GRAVES,TOT_HERIDOS_LEVES,TOT_VEHICULOS_IMPLICADOS,ZONA,ZONA_AGRUPADA,CARRETERA,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
711899,2016990000875,2015.0,12.0,11.0,4.0,28.0,13.0,999.0,28079.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,999,4.0,8.0,999.0,2.0,3.0,1.0,4.0,999.0,998,2.0
711900,2016990000876,2015.0,12.0,13.0,2.0,28.0,13.0,999.0,28079.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,999,4.0,8.0,1.0,999.0,1.0,1.0,1.0,999.0,999,7.0
711901,2016990001660,2015.0,11.0,15.0,1.0,28.0,13.0,999.0,28079.0,1.0,0.0,0.0,1.0,2.0,2.0,2.0,999,4.0,8.0,999.0,2.0,1.0,1.0,1.0,999.0,998,2.0


,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,TOT_MUERTOS,TOT_HERIDOS_GRAVES,TOT_HERIDOS_LEVES,TOT_VEHICULOS_IMPLICADOS,ZONA,ZONA_AGRUPADA,CARRETERA,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
418350,2012480000817,2012.0,10.0,9.0,1.0,48.0,16.0,999.0,999.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,A-8,5.0,7.0,2.0,999.0,3.0,1.0,4.0,1.0,998,19.0
357933,2012080001091,2012.0,1.0,15.0,2.0,8.0,9.0,999.0,8019.0,1.0,0.0,0.0,1.0,2.0,2.0,2.0,999,4.0,8.0,999.0,2.0,1.0,1.0,1.0,999.0,998,4.0
382601,2012170018821,2012.0,6.0,18.0,2.0,17.0,9.0,999.0,17117.0,1.0,0.0,0.0,1.0,2.0,2.0,2.0,999,4.0,8.0,999.0,2.0,3.0,1.0,4.0,999.0,998,2.0


{'Accvict': ['ID_ACCIDENTE',
  'ANIO',
  'MES',
  'HORA',
  'DIASEMANA',
  'PROVINCIA',
  'COMUNIDAD_AUTONOMA',
  'TOT_VICTIMAS',
  'TOT_MUERTOS',
  'TOT_HERIDOS_GRAVES',
  'TOT_HERIDOS_LEVES',
  'TOT_VEHICULOS_IMPLICADOS',
  'ZONA',
  'ZONA_AGRUPADA',
  'RED_CARRETERA',
  'TIPO_VIA',
  'SUPERFICIE_CALZADA',
  'LUMINOSIDAD',
  'FACTORES_ATMOSFERICOS',
  'VISIBILIDAD_RESTRINGIDA',
  'TIPO_ACCIDENTE']}

...

1    503603
3    121127
4     36137
2     33297
5     17738
Name: LUMINOSIDAD, dtype: int64

1    606046
4     58499
9     26100
5     11766
8      3519
3      2262
2      1430
7      1344
6       936
Name: FACTORES_ATMOSFERICOS, dtype: int64

1    421417
8    151022
0     94613
2     11739
7     11337
3     10176
4      7755
5      2681
6      1162
Name: VISIBILIDAD_RESTRINGIDA, dtype: int64

2     89924
4     82990
12    70554
7     55923
14    51057
3     34237
33    29470
20    27167
5     24207
13    22316
19    21280
16    21037
15    19466
71    15121
9     14116
1     13998
6     12085
68    10606
18    10454
11    10194
41     8905
10     7449
66     7166
64     5987
24     5595
56     4428
54     4363
63     4289
62     3068
52     2917
53     2890
61     2886
8      2387
67     2095
21     1853
51     1760
17     1732
65     1170
36     1091
57     1013
22      855
55      798
58      536
31      256
32       94
35       55
23       37
34       25
Name: TIPO_ACCIDENTE, dtype: int64

CREADA VISTA CON DATOS DE ACCIDENTES Y VISTIMAS DE 2008 A 2015


###   2. VEHICULOS

In [135]:
#Cabecera del fichero de Vehiculos
Header_Vehic=Headers_List[2]
Header_Vehic

,Vehic_2008,Vehic_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ID_VEHICULO,ID_VEHICULO
2,ANIO_MATRICULA_VEHICULO,ANIO_MATRICULA_VEHICULO
3,MES_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO
4,TIPO_VEHICULO,TIPO_VEHICULO
5,ESTADO_VEHICULO,NaN
6,NUMERO_OCUPANTES_VEH,NUMERO_OCUPANTES_VEH
7,MERCANCIAS_PELIGROSAS,MERCANCIAS_PELIGROSAS
8,VEHICULO_INCENDIADO,VEHICULO_INCENDIADO
9,ANIO,ANIO


In [136]:
#Matego solo las columnas inguales y actualizo mi diccionario con las cabeceras que utilizare en el proceso de carga

Header_Vehic=Header_Vehic[Header_Vehic['Vehic_2008']==Header_Vehic['Vehic_2015']]
Header_Vehic=Header_Vehic['Vehic_2008'].tolist()
dic_all_headers={'Vehic':Header_Vehic}
dic_all_headers

#Genero un df vacio donde concatenare todos los años para ver la cantidad de valores de que 
#dispongo por cada columna

df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Vehic'])

#Cargo cada csv y concateno
for file in dic_all_files['Vehic']:
    df_vehic=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Vehic'],encoding="ISO-8859-1")
    #relleno los nulos con 999
    df_vehic.fillna(999,inplace=True)
    df_all_years=pd.concat([df_all_years,df_vehic],ignore_index=True)
    
#muestreo los datos
df_all_years.head(3)
df_all_years.tail(3)
df_all_years.sample(3)

#De las columnas del data frame final, que contiene todos los años, 
#examino los distintos valores de las columnas, para tratar de decidir 
#que campos conservare para el analisis.


#Una vez examindas los resultado anteriores, decido descartar los siguientes campos de la vista:
df_all_years.drop(['MERCANCIAS_PELIGROSAS','VEHICULO_INCENDIADO'], axis=1, inplace=True)

#Actualizo mi diccionario con las cabeceras que utilizare en el proceso de carga
Header_Vehic=df_all_years.columns.tolist()
dic_all_headers={'Vehic':Header_Vehic}
dic_all_headers
#------------------SOBREECRITURA DE LOS DATOS APLICANDO MODIFICACIONES--------------
#Decido eliminor las siguientes columnas, por el alto numero de registro desconocidos y 
#porque considero que no me aportan valor:

#MERCANCIAS_PELIGROSAS
#VEHICULO_INCENDIADO

#Vacio el df de la vista, que contendra todos los años
df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Vehic'])

for file in dic_all_files['Vehic']:
    df_vehic=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Vehic'],encoding="ISO-8859-1")
    #relleno los nulos con 999
    df_vehic.fillna(999,inplace=True)
    for col in df_vehic.columns:
        if df_vehic[col].dtype !='O':
            df_vehic.ix[(df_vehic[col]==999), col]=df_vehic[col][df_vehic[col]!=999].median()


    df_vehic.to_csv(file,index=False,sep=";",encoding="ISO-8859-1")
    df_all_years=pd.concat([df_all_years,df_vehic],ignore_index=True)
    print("MODFICADO ----- "+file)

#Convierto tambien los numericos de la vista tipo float a enteros
#Compruebo que me ha sustituido correctamente los 999 por la mediana.
for col in df_all_years.columns:
        if df_all_years[col].dtype !='O':
            df_all_years[col]=df_all_years[col].astype(int)
            df_all_years[col].value_counts(dropna=False)

            
df_all_years.to_csv("./cleaned/view_vehic_08_15.csv",index=False,sep=";",encoding="ISO-8859-1")
print("CREADA VISTA CON DATOS DE ACCIDENTES Y VISTIMAS DE 2008 A 2015")
########   FIN DEL PROCESAMIENTO DE LOS DATASET DE VEHICULOS #########


{'Vehic': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ANIO_MATRICULA_VEHICULO',
  'MES_MATRICULA_VEHICULO',
  'TIPO_VEHICULO',
  'NUMERO_OCUPANTES_VEH',
  'MERCANCIAS_PELIGROSAS',
  'VEHICULO_INCENDIADO',
  'ANIO']}

,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,MERCANCIAS_PELIGROSAS,VEHICULO_INCENDIADO,ANIO
0,2.008410e+12,2.0,9999.0,99.0,22.0,1.0,N,N,2008.0
1,2.008520e+12,2.0,1991.0,99.0,22.0,1.0,N,N,2008.0
2,2.009460e+12,3.0,2004.0,1.0,22.0,1.0,N,N,2008.0


,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,MERCANCIAS_PELIGROSAS,VEHICULO_INCENDIADO,ANIO
1239590,2.016990e+12,1.0,2011.0,1.0,6.0,1.0,0,999,2015.0
1239591,2.016990e+12,1.0,9999.0,99.0,6.0,1.0,0,999,2015.0
1239592,2.016990e+12,2.0,2011.0,1.0,6.0,4.0,0,999,2015.0


,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,MERCANCIAS_PELIGROSAS,VEHICULO_INCENDIADO,ANIO
163636,2.008290e+12,1.0,2007.0,3.0,22.0,1.0,N,N,2008.0
260331,2.009090e+12,1.0,2004.0,12.0,22.0,4.0,N,N,2009.0
3439,2.008460e+12,2.0,1997.0,10.0,22.0,1.0,N,N,2008.0


{'Vehic': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ANIO_MATRICULA_VEHICULO',
  'MES_MATRICULA_VEHICULO',
  'TIPO_VEHICULO',
  'NUMERO_OCUPANTES_VEH',
  'ANIO']}

...

99    502516
1     161316
7      65918
6      61920
3      57533
5      55623
10     51713
4      51691
12     50245
11     48738
2      47150
9      43491
8      41739
Name: MES_MATRICULA_VEHICULO, dtype: int64

6     491756
22    300202
4     106634
2      67209
11     56248
9      43968
3      39686
43     26815
1      10572
5       9317
16      9295
10      9218
12      8412
41      6863
21      6253
15      6183
51      5666
20      5350
81      3928
55      3673
61      3244
52      2564
63      2335
13      2067
8       1917
90      1431
7       1301
23      1272
18      1125
14       866
42       813
82       628
31       529
30       403
32       402
24       291
53       231
19       227
17       187
62       165
70       159
54       148
80        40
Name: TIPO_VEHICULO, dtype: int64

1     946479
2     188288
3      48701
4      24477
0      18881
5       9476
6        969
7        455
8        251
9        212
10       181
20       133
15       115
11        77
30        73
12        72
25        43
14        42
16        42
40        42
13        41
18        37
50        36
21        32
22        25
17        24
23        24
19        23
35        23
27        21
       ...  
36        12
31        11
28        10
45        10
48         9
39         9
42         9
43         9
55         8
54         8
49         8
46         7
44         7
57         7
34         7
37         7
38         6
52         6
47         5
51         5
53         5
33         4
61         4
58         3
59         2
66         2
70         2
68         1
63         1
67         1
Name: NUMERO_OCUPANTES_VEH, dtype: int64

2015    170749
2008    163785
2014    158835
2013    155004
2009    154013
2010    149277
2011    144274
2012    143656
Name: ANIO, dtype: int64

CREADA VISTA CON DATOS DE ACCIDENTES Y VISTIMAS DE 2008 A 2015


###   1. PERSONAS

### TRATAMIENTO DATA SET PERSONAS

#### Se observa una diferencia en cuanto a la codificacion en los datos de sexo
#### entre los datasets anteriores y posteriores a 2011:

#####    -Para años anteriores a 2011 la codificacion es V o H
#####    -Para años posteriores, aparece codificados como 1 o 2, por lo que voy a intentar averiguar cual corresponde a hombre y mujeres
    
#### Es logico pensar que en los años anteriores a 1960, por ejemplo, el numero de hombres conductores era considerablemente al de mujeres, por lo que voy agrupar por edades y año de permiso de circulacion


#### Repito el procedimiento anterior para los datos desde 2011 a 2015 grabando los xls modificados en la carpeta cleaned junto con los otros data sets de años 2008 a 2010

### Repito el procedimiento anterior para los datos desde 2011 a 2015
### grabando los xls modificados en la carpeta cleaned junto con los otros data set 
### de años 2008 a 2010

In [137]:
#Cabecera del fichero de Personas
Header_Pers=Headers_List[1]
Header_Pers

,Pers_2008,Pers_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ID_VEHICULO,ID_VEHICULO
2,ID_PERSONA,ID_PERSONA
3,ID_CONDUCTOR,ID_CONDUCTOR
4,ID_PASAJERO,ID_PASAJERO
5,ID_PEATON,ID_PEATON
6,EDAD,EDAD
7,SEXO,SEXO
8,ANIO_PERMISO,ANIO_PERMISO
9,COD_POSICION,NaN


In [138]:
Header_Pers=Header_Pers[Header_Pers['Pers_2008']==Header_Pers['Pers_2015']]
Header_Pers=Header_Pers['Pers_2008'].tolist()
dic_all_headers={'Pers':Header_Pers}
dic_all_headers

df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Pers'])

#Cargo cada csv y concateno
for file in dic_all_files['Pers']:
    df_pers=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Pers'],encoding="ISO-8859-1")
    #relleno los nulos con 999
    df_pers=df_pers.fillna(999)
    df_all_years=pd.concat([df_all_years,df_pers],ignore_index=True)
    
#muestreo los datos
df_all_years.head(3)
df_all_years.tail(3)
df_all_years.sample(3)

#De las columnas del data frame final, que contiene todos los años, 
#examino los distintos valores de las columnas, para tratar de decidir 
#que campos conservare para el analisis.
for col in df_all_years.columns:
    #no saco los distinos valores para los id_accidente
    if col !='ID_ACCIDENTE':
        df_all_years[col].value_counts(dropna=False)

#Modifico el sexo y el año de permiso en cada fichero origen

for file in range(3,8):
    #leo fichero
    df_pers_sexo=pd.read_csv(dic_all_files['Pers'][file],delimiter=";",usecols=dic_all_headers['Pers'],encoding="ISO-8859-1")
    #Para sexo desconocido relleno aleatoriamente con 1 o 2
    df_pers_sexo[df_pers_sexo['SEXO']==999]=np.random.randint(1,2)

    #selecciono las personas que son conductores ('ID_PERSONA'=1) y los años desconocidos (9999) que quiero sutituir
    pers_grp=df_pers_sexo[(df_pers_sexo['ID_PERSONA']==1) & (df_pers_sexo['ANIO_PERMISO']!=9999)].groupby(['EDAD','SEXO'],as_index=False)['ANIO_PERMISO'].agg({'NUM_CONDUCTORES':'count','MEDIA_ANTIG_CARNET':'mean'})
    #conversion de a entero
    pers_grp['MEDIA_ANTIG_CARNET']=pers_grp['MEDIA_ANTIG_CARNET'].round(0)
    pers_grp['MEDIA_ANTIG_CARNET']=pers_grp['MEDIA_ANTIG_CARNET'].astype(int)

    #cruzo el df original por edad y sexo con el df con las medias
    merged=pd.merge(df_pers_sexo, pers_grp, how='left',on=['EDAD','SEXO'],suffixes=['_left', '_right'])
    merged.head()

    #1. sutituyo los 1 por V y los 2 por H
    merged.ix[(merged['SEXO']==1), 'SEXO'] ='V'
    merged.ix[(merged['SEXO']==2), 'SEXO'] ='M'
    #2. sustituyo los valores de años de permisos deconocidos
    merged.ix[((merged['ANIO_PERMISO']==9999) | (merged['ANIO_PERMISO']==0.0)) & (df_pers_sexo['ID_PERSONA']==1), 'ANIO_PERMISO'] =merged['MEDIA_ANTIG_CARNET']
    merged.head()
    #elimino del df las columnas con el numeor de conductores y la media de año de carnet
    merged.drop(['NUM_CONDUCTORES','MEDIA_ANTIG_CARNET'], axis=1, inplace=True)

    #si hubiese algun año NULL, lo relleno a 0.0
    merged['ANIO_PERMISO'].fillna(0.0,inplace=True)
    merged['ANIO_PERMISO']=merged['ANIO_PERMISO'].astype(int)

    #existen errores en los años de permiso, para conductores tenemos valores 0,1 y 999
    #muestro los diferentes valores de año de permiso
    pd.value_counts(merged['ANIO_PERMISO'])
    #elimino esos registros
    merged=merged[(merged['ANIO_PERMISO'].isin([0,1,999])) & merged['ID_PERSONA']!=1]
    #Antes de guarda cada fichero, sustituyo los 999 por la mediana en cada columna
    for col in merged.columns:
        if merged[col].dtype !='O':
            merged.ix[(merged[col]==999), col]=merged[col][merged[col]!=999].median()
    #guardo el fichero    
    merged.to_csv(dic_all_files['Pers'][file],index=False,sep=";",encoding="ISO-8859-1")
    print("MODFICADO ----- "+dic_all_files['Pers'][file])


#Genero la vista con los datos de todos los años
df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Pers'])

#Cargo de nuevo ya que he modificado las tablas que componen la vista y la genero de nuevo
for file in dic_all_files['Pers']:
    df_pers=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Pers'],encoding="ISO-8859-1")
    #relleno los nulos con 999
    df_pers.fillna(999,inplace=True)
    df_all_years=pd.concat([df_all_years,df_pers],ignore_index=True)
    
#Convierto tambien los numericos de la vista tipo float a enteros
#Compruebo que me ha sustituido correctamente los 999 por la mediana.
for col in df_all_years.columns:
        if df_all_years[col].dtype !='O':
            df_all_years.ix[(df_all_years[col]==999), col]=df_all_years[col][df_all_years[col]!=999].median()
            df_all_years[col]=df_all_years[col].astype(int)
            df_all_years[col].value_counts(dropna=False)
            
print("CREADA VISTA CON DATOS DE VEHICULOS DE 2008 A 2015")
df_all_years.to_csv("./cleaned/view_pers_08_15.csv",index=False,sep=";",encoding="ISO-8859-1")

{'Pers': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ID_PERSONA',
  'ID_CONDUCTOR',
  'ID_PASAJERO',
  'ID_PEATON',
  'EDAD',
  'SEXO',
  'ANIO_PERMISO',
  'MANIOBRAS',
  'INFRACC_VELOCIDAD',
  'INFRACC_PEATON',
  'ANIO']}

,ID_ACCIDENTE,ID_VEHICULO,ID_PERSONA,ID_CONDUCTOR,ID_PASAJERO,ID_PEATON,EDAD,SEXO,ANIO_PERMISO,MANIOBRAS,INFRACC_VELOCIDAD,INFRACC_PEATON,ANIO
0,"2,00801E+12",1.0,2.0,999.0,1.0,999.0,32.0,M,9999.0,999.0,999.0,999.0,2008.0
1,"2,00801E+12",2.0,2.0,999.0,1.0,999.0,60.0,M,9999.0,999.0,999.0,999.0,2008.0
2,"2,00801E+12",2.0,2.0,999.0,2.0,999.0,23.0,V,9999.0,999.0,999.0,999.0,2008.0


,ID_ACCIDENTE,ID_VEHICULO,ID_PERSONA,ID_CONDUCTOR,ID_PASAJERO,ID_PEATON,EDAD,SEXO,ANIO_PERMISO,MANIOBRAS,INFRACC_VELOCIDAD,INFRACC_PEATON,ANIO
1724733,2016990000358,999.0,3.0,999.0,999.0,1.0,31.0,1,999.0,999.0,999.0,2.0,2015.0
1724734,2016990000448,999.0,3.0,999.0,999.0,1.0,7.0,1,999.0,999.0,999.0,2.0,2015.0
1724735,2016990000876,999.0,3.0,999.0,999.0,2.0,7.0,2,999.0,999.0,999.0,2.0,2015.0


,ID_ACCIDENTE,ID_VEHICULO,ID_PERSONA,ID_CONDUCTOR,ID_PASAJERO,ID_PEATON,EDAD,SEXO,ANIO_PERMISO,MANIOBRAS,INFRACC_VELOCIDAD,INFRACC_PEATON,ANIO
1089141,2013080017135,1.0,1.0,1.0,999.0,999.0,32.0,1,2003.0,52.0,3.0,999.0,2013.0
1606767,201541091002758,1.0,1.0,1.0,999.0,999.0,32.0,1,2007.0,77.0,4.0,999.0,2015.0
1040156,2012390001396,999.0,3.0,999.0,999.0,1.0,6.0,1,999.0,999.0,999.0,1.0,2012.0


1.0      944267
2.0      573607
999.0     97957
3.0       78297
4.0       19402
5.0        5733
6.0        2203
7.0        1023
8.0         503
9.0         269
10.0        188
11.0        149
13.0        102
12.0         99
14.0         67
16.0         60
15.0         54
19.0         50
20.0         48
21.0         46
22.0         44
43.0         44
18.0         42
27.0         38
17.0         37
23.0         37
24.0         29
25.0         29
26.0         26
38.0         23
          ...  
57.0          1
56.0          1
91.0          1
58.0          1
73.0          1
89.0          1
88.0          1
87.0          1
85.0          1
84.0          1
83.0          1
82.0          1
81.0          1
92.0          1
76.0          1
75.0          1
71.0          1
59.0          1
70.0          1
69.0          1
68.0          1
67.0          1
66.0          1
65.0          1
93.0          1
63.0          1
64.0          1
62.0          1
61.0          1
60.0          1
Name: ID_VEHICULO, dtype

...

1     462907
18    461200
52    272067
72    130241
22     38214
21     32508
12     27529
71     26306
51     25656
3      24227
11     18053
2      12725
77     10759
0       8077
41      7446
43      7305
16      7112
19      7038
23      6856
20      6067
24      5891
17      5361
31      4296
15      4105
13      3492
5       3295
6       1784
25      1779
14      1748
29      1703
61      1574
7       1570
8       1252
42       954
30       701
4        604
9        586
26       499
10       174
27       121
Name: MANIOBRAS, dtype: int64

3    958538
4    452364
1     84546
5     82347
0     50616
2      5371
Name: INFRACC_VELOCIDAD, dtype: int64

9    1619355
0       5197
4       3303
8       2267
2       2036
1        813
5        652
6         92
7         57
3         10
Name: INFRACC_PEATON, dtype: int64

2008    223461
2015    217786
2009    212510
2010    206498
2014    205887
2013    200942
2012    187307
2011    179391
Name: ANIO, dtype: int64

CREADA VISTA CON DATOS DE VEHICULOS DE 2008 A 2015 EN ./CLEANED/----


### ESTAS SON LA COLUMNAS FINALES DE LOS 3 DATASETS

In [125]:
dic_all_headers={'Accvict':Header_Accvict,'Pers':Header_Pers,'Vehic':Header_Vehic}
dic_all_headers


{'Accvict': ['ID_ACCIDENTE',
  'ANIO',
  'MES',
  'HORA',
  'DIASEMANA',
  'PROVINCIA',
  'COMUNIDAD_AUTONOMA',
  'TOT_VICTIMAS',
  'TOT_MUERTOS',
  'TOT_HERIDOS_GRAVES',
  'TOT_HERIDOS_LEVES',
  'TOT_VEHICULOS_IMPLICADOS',
  'ZONA',
  'ZONA_AGRUPADA',
  'RED_CARRETERA',
  'TIPO_VIA',
  'SUPERFICIE_CALZADA',
  'LUMINOSIDAD',
  'FACTORES_ATMOSFERICOS',
  'VISIBILIDAD_RESTRINGIDA',
  'TIPO_ACCIDENTE'],
 'Pers': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ID_PERSONA',
  'ID_CONDUCTOR',
  'ID_PASAJERO',
  'ID_PEATON',
  'EDAD',
  'SEXO',
  'ANIO_PERMISO',
  'MANIOBRAS',
  'INFRACC_VELOCIDAD',
  'INFRACC_PEATON',
  'ANIO'],
 'Vehic': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ANIO_MATRICULA_VEHICULO',
  'MES_MATRICULA_VEHICULO',
  'TIPO_VEHICULO',
  'NUMERO_OCUPANTES_VEH',
  'ANIO']}

### -------------------------------------------------------------------------------------------------------
### PROCEDO A CARGAR EN DATAFRAMES DE CADA UNO DE LOS FICHEROS DEL DICCIONARIO
### -------------------------------------------------------------------------------------------------------

In [ ]:
df = pd.DataFrame({
    'x': [12, 20, 28, 18, 29, 33, 24, 45, 45, 52, 51, 52, 55, 53, 55, 61, 64, 69, 72],
    'y': [39, 36, 30, 52, 54, 46, 55, 59, 63, 70, 66, 63, 58, 23, 14, 8, 19, 7, 24]
})
df.head()
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(df)
labels = kmeans.predict(df)
centroids = kmeans.cluster_centers_
labels
centroids

In [ ]:
from matplotlib import pyplot as plt
from sklearn.datasets import load_iris

df = pd.read_csv('./cleaned/MICRODATOS_ACC_VICT_2015/TABLA_VEHIC_2015.csv',delimiter=";",encoding="ISO-8859-1")
df.head()

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(df)
labels = kmeans.predict(df)
centroids = kmeans.cluster_centers_
labels
centroids

fig = plt.figure(figsize=(5, 5))

#colors = map(lambda x: colmap[x+1], labels)

plt.scatter(df['ANIO_MATRICULA_VEHICULO'], df['MES_MATRICULA_VEHICULO'], alpha=0.5, edgecolor='k')

plt.xlim(0, 80)
plt.ylim(0, 80)
plt.show()


In [ ]:
import sqlite3 
import pandas as pd
with sqlite3.connect("./cleaned/jj.db") as con:
    df = pd.read_csv('./cleaned/view_vehic_08_15.csv',delimiter=";",encoding="ISO-8859-1")
    c = con.cursor()
    c.executescript('''
    CREATE TABLE f (
    ID_ACCIDENTE INTEGER,
    ID_VEHICULO INTEGER,
    ANIO_MATRICULA_VEHICULO INTEGER,
    MES_MATRICULA_VEHICULO INTEGER,
    TIPO_VEHICULO INTEGER,
    NUMERO_OCUPANTES_VEH INTEGER,
    ANIO INTEGER)''')
  
    df.to_sql('f', con, index=False,schema=False)
    con.commit

In [ ]:
import sqlite3
with sqlite3.connect("./cleaned/datos_dgt.db") as con:
    c = con.cursor()
    c.executescript(
    "CREATE TABLE Direcciones (calle TEXT, cp NUMERIC);"
    "INSERT INTO Direcciones VALUES ('Calle 1', 1644);"
    "INSERT INTO Direcciones VALUES ('Calle 2', 7594);"
    )
    c.executescript(
    "CREATE TABLE vehic;"
    )
    df = pandas.read_csv('./cleaned/view_vehic_08_15.csv',delimiter=";",encoding="ISO-8859-1")
    df.to_sql(vehic, con, index=False)
    con.commit
c.close

In [ ]:
#guardo las claves de cada tipo de fichero
list_keys_dic=dic_all_files.keys()
list_keys_dic
#Actualizo el diccionario con la rutas a los ficheros de trabajo
dic_all_files['Accvict'].append('./cleaned/view_accvict_08_15.csv')
dic_all_files['Pers'].append('./cleaned/view_pers_08_15.csv')
dic_all_files['Vehic'].append('./cleaned/view_vehic_08_15.csv')
dic_all_files

In [ ]:
dic_all_files['Accvict'][8]


In [ ]:
#Creo un dataframe vacio para posteriormente rellenarlo con los años 2001 al 2015
#Recorriendo por cada key los n values del diccionario

dic_total={}
df_total=pd.DataFrame()

for key in list_keys_dic:
    print ("#### "+key+" ####")
    for file in dic_all_files[key]:
        #meto en df cada file quedandome solo con las columnas seleccionadas anteriormente
        df_year=pd.read_csv(file,delimiter=";",encoding="ISO-8859-1",usecols=dic_all_headers[key])
        df_year=df_year.fillna(0)
        print("Cargando fichero-->"+file+"\n\t"+"Registros= "+str(df_year.shape[0])+"\n\t"+"Columnas= "+str(df_year.shape[1]))
        #df_total=df_total.append(df_year)
        df_total=pd.concat([df_total,df_year],ignore_index=True)
    dic_total[key] = df_total
print ("\n\t\t"+"##### CARGADOS EN DF_TOTAL TODOS LOS PERIODOS ########"+"\t\t")
df_total.shape

#### AHORA TENGO DE DATA FRAMES CON TODOS LOS AÑOS DESDE 2008 HASTA 2015

In [ ]:
##para categorizar
#dic_total['Accvict']['HORA']=dic_total['Accvict']['HORA'].astype('category')
#dic_total['Accvict']['HORA'].describe()
#este funciona
dic_total['Accvict']['HORA']= pd.Series(pd.Categorical(dic_total['Accvict']['HORA'], categories=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]))
dic_total['Accvict']['HORA'].cat.rename_categories(['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23'])
dic_total['Accvict']['HORA'].cat.codes

In [ ]:
#Extraigo los datos cuantitativos de victimas por
# 1. Año-mes-dia de la semana
# 2. Localizacion geografica

accvict_geo=pd.get_dummies(dic_total['Accvict'], prefix=['CA','PROV'], columns=['COMUNIDAD_AUTONOMA','PROVINCIA'])


In [ ]:
dic_total['Accvict']

In [ ]:
df = pd.DataFrame({'A': ['x', 'y', 'x'], 'B': ['z', 'u', 'z'],
                  'C': [1,2,3],
                  'D':['j', 'l', 'j']})

In [ ]:
df

In [ ]:
df_1= pd.get_dummies(df)
df_1

In [ ]:
df_2=pd.get_dummies(df, prefix=['DDDD','DDD','D'], columns=['A','B','C'])
df_2

In [ ]:
  df_year=pd.read_csv(files_list[file],delimiter=";",encoding="ISO-8859-1",converters={'COD_MUNICIPIO': str, 'MUNICIPIO': str})


### LECTURA DE LOS DATASET DE VEHICULOS PARA LOS AÑOS 2011-2015

In [ ]:
df_year.shape[0]

In [ ]:
#Creo un dataframe vacio para posteriormente rellenarlo con los años 2001 al 2015
df_accvict_08_15 = pd.DataFrame()
dict={}
for key in list_keys_dic:
    for file in range(0,len(files_list)):
        df_year=pd.read_csv(files_list[file],delimiter=";",encoding="ISO-8859-1")
        df_year.fillna(0)
        dict[tipology]=files_list[file]
        print("Cargando fichero-->"+files_list[file]+"\n\t"+"Registros= "+str(df_year.shape[0])+"\n\t"+"Columnas= "+str(df_year.shape[1]))
        if tipology=='list_accvict'
        %(tipology)+_08_15=df_accvict_08_15.append(df_year)


In [ ]:
j=df_accvict_08_15.groupby(['ANIO']).size()


In [ ]:
type(j)

In [ ]:
df_accvict_08_15.groupby(['ANIO']).agg(['mean', 'count'])

In [ ]:
print(df_accvict_08_15.head())
#df1=df[['ANIO','TOT_VICTIMAS']]


In [ ]:
dict

In [ ]:
threedee.scatter(df.ANIO, df["TOT_MUERTOS"], df['Volume'])